In [1]:
%pip install openai numpy matplotlib pydantic --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.


# Comparing Chat Completions & Responses APIs

This notebook explains the differences between two approaches for interacting with the OpenAI chat API:

1. **Chat Completions API (`chat.completions.create()`)**: A more *raw* interface where you provide a list of messages. In this model, you must handle the conversation state manually if you want to carry context from one turn to the next.

2. **Responses API (`responses.create()`)**: A newer, stateful API that not only lets you send messages but also automatically manages conversation state, making it easier to chain requests. This API is designed to be used with agents and will eventually replace the Assistants API.

Below, we'll break down the differences in arguments and design, and then look at a minimal code example for each.

## Key Differences

- **State Management**:
  - **Chat Completions**: You provide a `messages` array; every call is stateless unless you manually add previous messages.
  - **Responses API**: Automatically manages conversation state. You can chain requests easily using parameters like `store` and `previous_response_id`.

- **Arguments & Structure**:
  - **Chat Completions**: Mainly accepts a `messages` array with objects having keys like `role` and `content`. It returns results in a `choices` list, where you get the output from `message.content`.
  - **Responses API**: Uses an `input` parameter (or an array of message objects) and returns a structured `output` which aggregates results. It also provides additional state management features.

- **Future Direction**:
  - The Responses API is designed for building agents and will replace the current Assistants API. It offers a smoother developer experience by holding the chat context between turns.

In summary, while the Chat Completions endpoint gives you direct control but requires manual state management, the Responses API simplifies multi-turn interactions and is better suited for advanced applications.

## Minimal Code Example: Chat Completions API

This example shows a typical call to `chat.completions.create()`. Note that if you want to carry context from one call to the next, you must manually append previous messages.

In [1]:
from openai import OpenAI

MODEL = "gpt-4.1-mini"

In [2]:
# Initialize the client
# Note: In a real application, you would use an environment variable or secure method
# to store your API key. This is just for demonstration.
client = OpenAI(
    # Replace with your actual API key or use: api_key=os.environ.get("OPENAI_API_KEY")
    api_key="YOUR_API_KEY_HERE"
)

In [3]:
# A single-turn conversation using chat completions
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Tell me a joke."}
    ]
)

print("Chat Completions response:", response.choices[0].message.content)

# For multiple turns, you would manually append previous messages:
conversation = [
    {"role": "user", "content": "Tell me a joke."},
    {"role": "assistant", "content": response.choices[0].message.content}
]

# Follow-up request
followup = client.chat.completions.create(
    model=MODEL,
    messages=conversation + [{"role": "user", "content": "Tell me another joke."}]
)

print("Follow-up response:", followup.choices[0].message.content)

Chat Completions response: Why did the scarecrow win an award? 

Because he was outstanding in his field!
Follow-up response: Why don't skeletons fight each other? 

They don't have the guts!


## Minimal Code Example: Responses API

This example uses the Responses API. Notice that it uses the `input` parameter instead of `messages` and can easily chain conversation state—this makes it well suited for agent applications.

In [4]:
# A single-turn conversation using the Responses API
response = client.responses.create(
    model=MODEL,
    input=[{"role": "user", "content": "Tell me a joke."}]
)

print("Responses API response:", response.output_text)

# Chaining conversation state automatically (the state is held internally):
second_response = client.responses.create(
    model=MODEL,
    previous_response_id=response.id,
    input=[{"role": "user", "content": "Tell me another joke."}]
)

print("Chained response:", second_response.output_text)

Responses API response: Sure! Here's one for you:

Why don’t scientists trust atoms?

Because they make up everything!
Chained response: Of course! Here’s another one:

Why did the scarecrow win an award?

Because he was outstanding in his field!


## Summary

- **Chat Completions API:**
  - Requires manual handling of conversation state (append messages by yourself).
  - Uses the `messages` parameter and returns output under `choices` → `message.content`.

- **Responses API:**
  - Manages conversation state automatically, easing the chaining of multi-turn interactions.
  - Uses the `input` parameter and returns aggregated output as `output_text`.
  - Will be the API of choice for building agents and is set to replace the current Assistants API.

By choosing between these two, you can decide whether you prefer raw control (and manual state management) or a streamlined, stateful approach for more complex, agent-like interactions.